# EPyR Tools - Advanced EPR Analysis\n\nThis notebook demonstrates advanced analysis techniques for EPR spectroscopy using EPyR Tools.\n\n## What You'll Learn\n\n- Peak detection and fitting in EPR spectra\n- g-factor calculations and magnetic field calibration\n- Spectral integration for quantitative analysis\n- Hyperfine structure analysis\n- Data export for publication\n\n## Background\n\nAdvanced EPR analysis involves extracting quantitative information from spectra:\n- **g-factors**: Electronic environment and orbital contributions\n- **Hyperfine coupling**: Nuclear-electron interactions\n- **Linewidths**: Relaxation processes and dynamics\n- **Concentrations**: Spin quantification through integration

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize, integrate, signal
from pathlib import Path
import warnings

# Add EPyR Tools to path
epyr_root = Path().resolve().parent.parent
if str(epyr_root) not in sys.path:
    sys.path.insert(0, str(epyr_root))

import epyr.eprload as eprload
from epyr.baseline import baseline_polynomial
import epyr.constants as const

# Suppress minor warnings for cleaner output
warnings.filterwarnings('ignore', category=RuntimeWarning)

print("EPyR Tools advanced analysis module loaded successfully!")
print(f"Physical constants available: h={const.h:.2e}, mu_B={const.mu_B:.2e}, g_e={const.g_e:.6f}")

## 1. Load and Prepare EPR Data\n\nWe'll create synthetic EPR data for demonstration or load real data if available.

In [ ]:
# Try to load real EPR data first\ndata_dir = Path(\"../data\")\nsample_file = None\n\n# Look for sample files\nfor file_path in (data_dir / \"BES3T\").glob(\"*.dsc\"):\n    sample_file = file_path\n    break\n\nif not sample_file:\n    for file_path in (data_dir / \"ESP\").glob(\"*.par\"):\n        sample_file = file_path\n        break\n\nif sample_file:\n    print(f\"Loading real EPR data: {sample_file.name}\")\n    x, y, params, filepath = eprload.eprload(str(sample_file), plot_if_possible=False)\n    \n    if x is None or y is None:\n        print(\"Failed to load real data. Using synthetic data instead.\")\n        sample_file = None\n    else:\n        # Apply baseline correction\n        y, baseline_fit = baseline_polynomial(y, x_data=x, poly_order=1)\n        freq_ghz = params.get('MWFQ', params.get('MF', 9.4e9))\n        if isinstance(freq_ghz, str):\n            freq_ghz = float(freq_ghz)\n        freq_ghz = freq_ghz / 1e9 if freq_ghz > 1e6 else freq_ghz\n        data_source = \"real\"\n\n# Create synthetic data with realistic baseline issues\nif sample_file is None:\n    print(\"Creating synthetic nitroxide EPR spectrum for analysis...\")\n    \n    # Generate field axis\n    x = np.linspace(3300, 3400, 2000)  # High resolution for analysis\n    freq_ghz = 9.4  # X-band frequency\n    \n    # Nitroxide parameters (typical TEMPO)\n    g_iso = 2.006\n    A_N = 16.0  # Gauss, nitrogen hyperfine\n    linewidth = 1.2  # Gauss, intrinsic linewidth\n    \n    # Calculate field positions for three nitrogen lines\n    center_field = freq_ghz * 1e9 * const.h / (g_iso * const.mu_B) * 1e4\n    field_positions = [center_field - A_N, center_field, center_field + A_N]\n    intensities = [1.0, 1.0, 1.0]  # Equal intensities for I=1 nitrogen\n    \n    # Generate spectrum as sum of Lorentzian lines\n    y = np.zeros_like(x)\n    for pos, intensity in zip(field_positions, intensities):\n        # Lorentzian lineshape (more realistic for solution EPR)\n        y += intensity * (linewidth/2)**2 / ((x - pos)**2 + (linewidth/2)**2)\n    \n    # Convert to first derivative (typical EPR display)\n    y = np.gradient(y, x)\n    \n    # Add realistic noise\n    noise_level = np.max(np.abs(y)) * 0.02\n    y += np.random.normal(0, noise_level, len(x))\n    \n    data_source = \"synthetic\"\n    \n    print(f\"Created synthetic nitroxide spectrum:\")\n    print(f\"  Frequency: {freq_ghz:.1f} GHz\")\n    print(f\"  g-factor: {g_iso:.3f}\")\n    print(f\"  A_N: {A_N:.1f} G\")\n    print(f\"  Linewidth: {linewidth:.1f} G\")\n\nprint(f\"\\nData ready for analysis: {data_source} spectrum with {len(x)} points\")\nprint(f\"Field range: {x.min():.1f} - {x.max():.1f} G\")\nprint(f\"Microwave frequency: {freq_ghz:.3f} GHz\")

In [ ]:
# Visualize the data for analysis\nplt.figure(figsize=(12, 6))\nplt.plot(x, y, 'b-', linewidth=1.5, label='EPR Spectrum')\nplt.xlabel('Magnetic Field (G)')\nplt.ylabel('EPR Signal (a.u.)')\nplt.title(f'EPR Spectrum for Analysis - {freq_ghz:.1f} GHz')\nplt.legend()\nplt.grid(True, alpha=0.3)\nplt.show()\n\n# Basic spectral statistics\nprint(\"\\nSpectral Overview:\")\nprint(f\"Signal range: {np.min(y):.3f} to {np.max(y):.3f}\")\nprint(f\"Peak-to-peak: {np.ptp(y):.3f}\")\nprint(f\"Signal-to-noise ratio: {np.ptp(y) / np.std(y):.1f}\")

## 2. Peak Detection and Analysis\n\nLet's identify the main features in the EPR spectrum.

In [ ]:
# Peak detection in EPR spectra\nprint(\"Detecting EPR spectral features...\")\n\n# For first-derivative EPR spectra, we look for zero-crossings and extrema\ndef find_epr_peaks(x, y, threshold_factor=0.1):\n    \"\"\"Find peaks in EPR first-derivative spectrum.\"\"\"\n    \n    # Find zero crossings (absorption peak centers)\n    zero_crossings = []\n    for i in range(1, len(y)):\n        if y[i-1] * y[i] < 0:  # Sign change\n            # Linear interpolation to find exact crossing\n            if abs(y[i] - y[i-1]) > 1e-10:  # Avoid division by zero\n                x_cross = x[i-1] - y[i-1] * (x[i] - x[i-1]) / (y[i] - y[i-1])\n                zero_crossings.append(x_cross)\n    \n    # Find extrema (derivative peaks)\n    threshold = np.max(np.abs(y)) * threshold_factor\n    \n    # Positive peaks (upward parts of derivative)\n    peaks_pos, _ = signal.find_peaks(y, height=threshold, distance=5)\n    \n    # Negative peaks (downward parts of derivative)\n    peaks_neg, _ = signal.find_peaks(-y, height=threshold, distance=5)\n    \n    return zero_crossings, peaks_pos, peaks_neg\n\nzero_crossings, peaks_pos, peaks_neg = find_epr_peaks(x, y)\n\nprint(f\"Found {len(zero_crossings)} absorption peaks (zero crossings)\")\nprint(f\"Found {len(peaks_pos)} positive derivative peaks\")\nprint(f\"Found {len(peaks_neg)} negative derivative peaks\")\n\n# Display peak positions\nif zero_crossings:\n    print(\"\\nAbsorption peak positions (G):\")\n    for i, pos in enumerate(zero_crossings):\n        print(f\"  Peak {i+1}: {pos:.2f} G\")\n        \n    # Calculate splittings if multiple peaks\n    if len(zero_crossings) > 1:\n        splittings = np.diff(zero_crossings)\n        print(f\"\\nHyperfine splittings (G):\")\n        for i, split in enumerate(splittings):\n            print(f\"  Δ{i+1}-{i+2}: {split:.2f} G\")\n        \n        if len(splittings) > 1:\n            print(f\"  Average splitting: {np.mean(splittings):.2f} ± {np.std(splittings):.2f} G\")

In [ ]:
# Visualize peak detection results\nplt.figure(figsize=(12, 8))\n\nplt.plot(x, y, 'b-', linewidth=1.5, label='EPR spectrum')\n\n# Mark zero crossings (absorption peaks)\nif zero_crossings:\n    for i, pos in enumerate(zero_crossings):\n        plt.axvline(x=pos, color='red', linestyle='--', alpha=0.7)\n        plt.text(pos, plt.ylim()[1]*0.9, f'Peak {i+1}\\n{pos:.1f} G', \n                ha='center', va='top', fontsize=9,\n                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))\n\n# Mark derivative extrema\nif len(peaks_pos) > 0:\n    plt.plot(x[peaks_pos], y[peaks_pos], 'ro', markersize=6, label='Positive peaks')\nif len(peaks_neg) > 0:\n    plt.plot(x[peaks_neg], y[peaks_neg], 'go', markersize=6, label='Negative peaks')\n\nplt.xlabel('Magnetic Field (G)')\nplt.ylabel('EPR Signal (a.u.)')\nplt.title('EPR Peak Detection Results')\nplt.legend()\nplt.grid(True, alpha=0.3)\nplt.show()\n\n# Analysis summary\nprint(\"\\nPeak Analysis Summary:\")\nprint(\"=\" * 30)\nif len(zero_crossings) == 1:\n    print(\"Single absorption peak detected - likely organic radical or transition metal\")\nelif len(zero_crossings) == 3:\n    print(\"Three-line pattern detected - likely nitroxide radical (I=1 nitrogen)\")\nelif len(zero_crossings) == 2:\n    print(\"Two-line pattern detected - possible doublet splitting\")\nelif len(zero_crossings) > 3:\n    print(f\"{len(zero_crossings)}-line pattern detected - complex hyperfine structure\")\nelse:\n    print(\"No clear peaks detected - may need different analysis approach\")

## 3. g-Factor Calculations\n\nThe g-factor is fundamental for EPR analysis.

In [ ]:
# Calculate g-factors for detected peaks\nprint(\"Calculating g-factors...\")\n\ndef calculate_g_factor(field_gauss, frequency_hz):\n    \"\"\"Calculate g-factor from field and frequency.\"\"\"\n    g = const.h * frequency_hz / (const.mu_B * field_gauss * 1e-4)\n    return g\n\nfreq_hz = freq_ghz * 1e9\n\nprint(f\"Microwave frequency: {freq_ghz:.6f} GHz ({freq_hz:.0f} Hz)\")\nprint(f\"Free electron g-factor: {abs(const.g_e):.6f}\")\nprint()\n\ng_factors = []\nif zero_crossings:\n    print(\"Peak positions and g-factors:\")\n    print(f\"{'Peak':<8} {'Field (G)':<12} {'g-factor':<12} {'Δg':<12}\")\n    print(\"-\" * 48)\n    \n    for i, field in enumerate(zero_crossings):\n        g = calculate_g_factor(field, freq_hz)\n        delta_g = g - abs(const.g_e)\n        g_factors.append(g)\n        \n        print(f\"{i+1:<8} {field:<12.3f} {g:<12.6f} {delta_g:<+12.6f}\")\n    \n    # Statistical analysis of g-factors\n    if len(g_factors) > 1:\n        g_mean = np.mean(g_factors)\n        g_std = np.std(g_factors)\n        g_range = np.ptp(g_factors)\n        \n        print(f\"\\nStatistical Analysis:\")\n        print(f\"Mean g-factor: {g_mean:.6f} ± {g_std:.6f}\")\n        print(f\"g-factor range: {g_range:.6f}\")\n        print(f\"g-anisotropy: {g_range/g_mean*100:.3f}%\")\n        \n        # Check if isotropic (small g-spread) or anisotropic\n        if g_range < 0.001:\n            print(\"→ Isotropic system (solution or narrow powder pattern)\")\n        elif g_range < 0.01:\n            print(\"→ Weakly anisotropic system\")\n        else:\n            print(\"→ Strongly anisotropic system (powder pattern)\")\n    \n    else:\n        g = g_factors[0]\n        delta_g = g - abs(const.g_e)\n        print(f\"\\nSingle peak analysis:\")\n        print(f\"g-factor: {g:.6f} (Δg = {delta_g:+.6f})\")\n        \n        # Interpret g-value\n        if abs(delta_g) < 0.0005:\n            print(\"→ Near free electron g-value: organic radical\")\n        elif delta_g > 0.002:\n            print(\"→ g > g_e: possible transition metal or heteroatom\")\n        else:\n            print(\"→ Slightly shifted g-value: organic radical with some heterogeneity\")\n\nelse:\n    print(\"No peaks detected for g-factor calculation\")\n    # Try to estimate from spectrum center\n    center_idx = len(x) // 2\n    center_field = x[center_idx]\n    g_estimate = calculate_g_factor(center_field, freq_hz)\n    print(f\"Estimated g-factor from spectrum center: {g_estimate:.6f}\")

## 4. Spectral Integration\n\nIntegration provides quantitative information about spin concentrations.

In [ ]:
# Single and double integration of EPR spectrum\nprint(\"Performing spectral integration...\")\n\n# Single integration (first derivative → absorption)\ndx = x[1] - x[0]  # Field step\nabsorption = np.cumsum(y) * dx\n\n# Remove baseline drift from integrated spectrum\nabsorption_corrected = absorption - np.linspace(absorption[0], absorption[-1], len(absorption))\n\n# Double integration (absorption → double integral)\ndouble_integral = np.cumsum(absorption_corrected) * dx\n\n# Calculate total spin count (proportional to double integral area)\ntotal_double_integral = double_integral[-1] - double_integral[0]\n\nprint(f\"Field step: {dx:.4f} G\")\nprint(f\"Single integral range: {absorption_corrected.min():.3f} to {absorption_corrected.max():.3f}\")\nprint(f\"Double integral total: {total_double_integral:.3e}\")\n\n# Estimate spin concentration (requires calibration)\nprint(\"\\nSpin Quantification:\")\nprint(\"Note: Absolute concentrations require calibration with known standards\")\n\n# Relative quantification based on double integral\nif abs(total_double_integral) > 0:\n    print(f\"Relative spin count: {abs(total_double_integral):.3e} arbitrary units\")\nelse:\n    print(\"Zero total double integral - check spectrum baseline\")

In [ ]:
# Visualize integration results\nplt.figure(figsize=(14, 10))\n\n# Original first derivative spectrum\nplt.subplot(3, 1, 1)\nplt.plot(x, y, 'b-', linewidth=1.5, label='First derivative (original)')\nplt.xlabel('Magnetic Field (G)')\nplt.ylabel('dχ\\\"/dB (a.u.)')\nplt.title('EPR Spectrum - First Derivative')\nplt.legend()\nplt.grid(True, alpha=0.3)\n\n# Single integral (absorption)\nplt.subplot(3, 1, 2)\nplt.plot(x, absorption, 'gray', linewidth=1, alpha=0.7, label='Raw integral')\nplt.plot(x, absorption_corrected, 'r-', linewidth=2, label='Absorption (baseline corrected)')\nplt.xlabel('Magnetic Field (G)')\nplt.ylabel('χ\" (a.u.)')\nplt.title('Single Integration - Absorption Spectrum')\nplt.legend()\nplt.grid(True, alpha=0.3)\n\n# Double integral\nplt.subplot(3, 1, 3)\nplt.plot(x, double_integral, 'g-', linewidth=2, label='Double integral')\nplt.axhline(y=0, color='k', linestyle=':', alpha=0.5)\nplt.xlabel('Magnetic Field (G)')\nplt.ylabel('∫∫ Signal (a.u.)')\nplt.title('Double Integration - Proportional to Spin Count')\nplt.legend()\nplt.grid(True, alpha=0.3)\n\n# Add integration statistics as text\nstats_text = f'Total Double Integral: {total_double_integral:.3e}\\n'\nstats_text += f'Peak Absorption: {np.max(np.abs(absorption_corrected)):.3f}'\n\nplt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,\n         verticalalignment='top', fontsize=10,\n         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))\n\nplt.tight_layout()\nplt.show()

## 5. Summary and Export\n\nLet's summarize all analysis results.

In [ ]:
# Compile comprehensive analysis results\nprint(\"EPR ANALYSIS SUMMARY\")\nprint(\"=\" * 50)\n\nprint(f\"\\n📊 MEASUREMENT PARAMETERS:\")\nprint(f\"   Microwave frequency: {freq_ghz:.3f} GHz\")\nprint(f\"   Field range: {x.min():.1f} - {x.max():.1f} G\")\nprint(f\"   Resolution: {(x[1] - x[0]):.4f} G/point\")\nprint(f\"   Data points: {len(x):,}\")\n\nprint(f\"\\n🎯 PEAK ANALYSIS:\")\nn_peaks = len(zero_crossings)\nif n_peaks > 0:\n    print(f\"   Number of absorption peaks: {n_peaks}\")\n    for i, pos in enumerate(zero_crossings):\n        print(f\"   Peak {i+1}: {pos:.2f} G\")\n    \n    if len(zero_crossings) > 1:\n        splittings = np.diff(zero_crossings)\n        avg_split = np.mean(splittings)\n        print(f\"   Average splitting: {avg_split:.2f} G\")\nelse:\n    print(f\"   No distinct peaks detected\")\n\nprint(f\"\\n⚛️  G-FACTOR ANALYSIS:\")\nif g_factors:\n    mean_g = np.mean(g_factors)\n    delta_g = mean_g - abs(const.g_e)\n    print(f\"   Mean g-factor: {mean_g:.6f}\")\n    print(f\"   Δg (g - |g_e|): {delta_g:+.6f}\")\n    \n    if len(g_factors) > 1:\n        g_std = np.std(g_factors)\n        print(f\"   g-factor spread: ±{g_std:.6f}\")\nelse:\n    print(f\"   g-factor calculation not available\")\n\nprint(f\"\\n📈 QUANTITATIVE ANALYSIS:\")\nprint(f\"   Double integral: {total_double_integral:.3e} (∝ spin count)\")\n\nprint(f\"\\n🔍 SPECTRAL QUALITY:\")\nsnr = np.ptp(y) / np.std(y)\nptp = np.ptp(y)\nprint(f\"   Signal-to-noise ratio: {snr:.1f}\")\nprint(f\"   Peak-to-peak amplitude: {ptp:.3f}\")\n\nprint(f\"\\n✅ ADVANCED EPR ANALYSIS COMPLETE!\")\nprint(f\"\\nNext steps:\")\nprint(f\"• Compare with literature EPR values\")\nprint(f\"• Use results for quantitative EPR analysis\")\nprint(f\"• Process additional spectra for comparison\")